<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Start: Import libraries


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# If the spacex_launch_geo.csv is not available in local, you can download it from IBM server: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv
#spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

spacex_df=pd.read_csv('../datasets/spacex_launch_geo.csv')

In [5]:
spacex_df.columns

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, 
                       radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#54102F;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [59]:
# Initial the map
mid_point = [20.385825, -100.710019]
site_map = folium.Map(location=mid_point, zoom_start=4.4)

# Create a MarkerCluster to group markers that are close together
marker_cluster = MarkerCluster().add_to(site_map)

for i in range(launch_sites_df.shape[0]):
    site = launch_sites_df.iloc[i]
    coord = [site['Lat'], site['Long']]
    
    # Add a circle for the launch site
    circle = folium.Circle(
        coord, 
        radius=1000, 
        color='#d35400', 
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))
    site_map.add_child(circle)
    
    # Create a marker with a text label
    marker = folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(100,36),
            icon_anchor=(20,0),  # adjust the anchor point if needed
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site']
        )
    )
    # Add the marker to the MarkerCluster instead of the base map
    marker_cluster.add_child(marker)

equator_line = folium.PolyLine(locations=[[0, -180], [0, 180]], color='blue', weight=2, dash_array='5, 5')
site_map.add_child(equator_line)

site_map


To think a bit:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

There are 4 launch sites with no proximity to the Equator line, and 3 of them are very close to the coast and close to each other. All of the launch sites are close to the coast. The importance of being close to the coast is that the rockets can be launched over the ocean, which is safer. The reason for the proximity to the Equator line is that the rotational speed of the Earth is faster at the Equator line, which can help the rockets to reach orbit faster. However, the disadvantage of being close to the Equator line is that the rockets need to travel further to reach the orbit, which requires more fuel. Anyways, the proximity of all launch sites to the Equator is not very close, and the difference in distance to the Equator is not very significant.

## Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [60]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [61]:
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [53]:
# Apply a function to check the value of `class` column
def ColumnClassCheck(class_column):
    '''This function returns a list color based on the input value given a 2-class classification'''
    # If class=1, marker_color value will be green
    # If class=0, marker_color value will be red
    class_column = class_column.tolist()
    
    color_class = []

    for i in range(0, len(class_column)):
        if class_column[i] == 1:
            marker_color = 'green'
            color_class.append(marker_color)
        else:
            marker_color = 'red'
            color_class.append(marker_color)
    
    return color_class

In [54]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    

In [55]:
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [62]:
#Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

#For each row in spacex_df data frame create a Marker object with its coordinate and customize 
#the Marker's icon property to indicate if this launch was successed or failed
for index, record in spacex_df.iterrows():
    #Create the marker using the location, adding a popup and the icon
    marker = folium.Marker(
        location=[record['Lat'], record['Long']], 
        tooltip="Yeih!" if record['class'] == 1 else "Booo!",
        icon=folium.Icon(color='white', 
                        icon_color=record['marker_color']
        )
    )
    #Add the marker to the cluster
    marker_cluster.add_child(marker)

#Display the map
site_map

## Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [63]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [64]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [32]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
#distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

#Iterate over all launch sites that are not VAFB SLC-4E and calculate the distance between the launch site and the coastline
for i in range(launch_sites_df.shape[0]):
    site = launch_sites_df.iloc[i]
    launch_site_lat = site['Lat']
    launch_site_lon = site['Long']

    if site['Launch Site'] != 'VAFB SLC-4E':
        distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, 28.56367, -80.57163)
        print(f"Distance from Launch Site {site['Launch Site']} to Cape Canaveral is {distance_coastline} km")

Distance from Launch Site CCAFS LC-40 to Cape Canaveral is 0.5797581813109574 km
Distance from Launch Site CCAFS SLC-40 to Cape Canaveral is 0.5097439631188213 km
Distance from Launch Site KSC LC-39A to Cape Canaveral is 7.429343134068913 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
coordinate = [28.563567, -80.568028] #NOTE: Just an example, this is NOT the actual coordinate closest to the selected launch site
distance = calculate_distance(launch_sites_df.iloc[1]['Lat'], launch_sites_df.iloc[1]['Long'], coordinate[0], coordinate[1])

distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

site_map.add_child(distance_marker)

Draw a `PolyLine` between a launch site to the selected coastline point


In [66]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
lines=folium.PolyLine(locations=[coordinate, [launch_sites_df.iloc[1]['Lat'], launch_sites_df.iloc[1]['Long']]], weight=1)

site_map.add_child(lines) #NOTE: THE IDEA WORKS, IT'S NEEDED TO REPLACE THE COORDINATES TO THE CORRECT ONES

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


Similarly, we could draw a line betwee a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [67]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [68]:
#Import the cities, highways and railways data extracted from https://geojson.io/
cities = pd.read_csv('../datasets/points-cities.csv')
highways = pd.read_csv('../datasets/points-highways.csv')
railways = pd.read_csv('../datasets/points-railways.csv')

In [69]:
cities.dtypes

name     object
lon     float64
lat     float64
dtype: object

In [87]:
#The format of the data is as follows: point, longitude, latitude

#Iterate over the cities data and add a folium.Marker for each city
for i in range(cities.shape[0]):
    city = cities.iloc[i]
    coord = [city['lat'], city['lon']]
    
    marker = folium.Marker(
        coord,
        icon=folium.Icon(color='blue')
    )
    
    site_map.add_child(marker)

#Show the map
site_map

In [88]:
highways

,highway,lon,lat
0,Cape Canaveral,-80.580607,28.493741
1,Lompoc,-120.457652,34.638999


In [89]:
#Add a marker cluster to the map for the highways data
marker_cluster_highways = MarkerCluster()

#Iterate over the highways data and add a folium.Marker for each highway
for i in range(highways.shape[0]):
    highway = highways.iloc[i]
    coord = [highway['lat'], highway['lon']]
    
    marker = folium.Marker(
        coord,
        #Use a DivIcon to add a custom icon with the name of the highway
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % highway['highway'],
        )
    )
    
    marker_cluster_highways.add_child(marker)

#Iterate over the highways data and add a folium.PolyLine for each highway from each lunch site
for i in range(launch_sites_df.shape[0]):
    site = launch_sites_df.iloc[i]
    for j in range(highways.shape[0]):
        highway = highways.iloc[j]
        lines = folium.PolyLine(locations=[[site['Lat'], site['Long']], [highway['lat'], highway['lon']]], weight=1)
        site_map.add_child(lines)

#Show the map
site_map

In [90]:
#Iterate over all cities and launch sites and calculate the distance between them and save the result in a list
distances = []
for i in range(cities.shape[0]):
    city = cities.iloc[i]
    for j in range(launch_sites_df.shape[0]):
        site = launch_sites_df.iloc[j]
        distance = calculate_distance(city['lat'], city['lon'], site['Lat'], site['Long'])
        distances.append([city['name'], site['Launch Site'], distance])

#Draw a line between each city and launch site in red color and the distance on the line
for i in range(len(distances)):
    distance = distances[i]
    city = cities[cities['name'] == distance[0]]
    site = launch_sites_df[launch_sites_df['Launch Site'] == distance[1]]
    lines = folium.PolyLine(locations=[[city['lat'].values[0], city['lon'].values[0]], [site['Lat'].values[0], site['Long'].values[0]]], weight=1, color='red')
    site_map.add_child(lines)
    marker = folium.Marker(
        [(city['lat'].values[0] + site['Lat'].values[0]) / 2, (city['lon'].values[0] + site['Long'].values[0]) / 2],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance[2]),
        )
    )
    site_map.add_child(marker)

#Show the map
site_map

In [91]:
railways

,railway,lon,lat
0,Vandenberg,-120.623901,34.636180
1,Titan3,-80.582950,28.583195
2,Titan2,-80.583876,28.583696
3,Titan4,-80.582118,28.582796
4,Titan1,-80.590953,28.583845


In [92]:
#Iterate over the railways data and add a folium.Marker for each railway as a DivIcon
for i in range(railways.shape[0]):
    railway = railways.iloc[i]
    coord = [railway['lat'], railway['lon']]
    
    marker = folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % railway['railway'],
        )
    )
    
    site_map.add_child(marker)

#Draw a pink line between each launch site and railway
for i in range(launch_sites_df.shape[0]):
    site = launch_sites_df.iloc[i]
    for j in range(railways.shape[0]):
        railway = railways.iloc[j]
        lines = folium.PolyLine(locations=[[site['Lat'], site['Long']], [railway['lat'], railway['lon']]], weight=1, color='pink')
        site_map.add_child(lines)
    
#Show the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


## Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
